# Trabalho 7
## Rede Madaline para reconhecimento de dígitos
<img src="digitos.png" alt="digitos" width="250"/>

In [1]:
import os
import random
from collections import defaultdict

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from ipywidgets import *
import ipywidgets as widgets

from IPython.display import display, HTML

########## entradas dos parametros ##########
style = {'description_width': 'initial'}

taxa_aprendizagem = widgets.FloatSlider(
    value=0.5,min=0.001,max=1,step=0.001,
    description='Taxa de aprendizagem:',
    continuous_update=False,
    readout=True, style=style
)
display(taxa_aprendizagem)

parada = widgets.RadioButtons(
    options=['Erro', 'Ciclos'],
    description='Critério de parada:',
    disabled=False, style=style
)
display(parada)

ciclos = widgets.IntSlider(
    value=10,min=1,max=50,step=1,
    description='Número de ciclos:',
    continuous_update=False,
    readout=True, style=style
)
display(ciclos)

erro = widgets.FloatSlider(
    value=0.1,min=0,max=100,step=0.001,
    description='Erro tolerado:',
    continuous_update=False,
    readout=True, style=style
)
display(erro)
##############################################

##### conversao das letras em inputs.csv #####
def inputs_to_csv():
    #os.getcwd()
    path = 'trab7/letras'
    files = [f for f in os.listdir(path) if not f.startswith('.')]
    files.sort()
    letters = []
    for file_name in files:
        file = pd.read_csv("{}/{}".format(path, file_name), delim_whitespace=True, header=None)
        letters.append(file.values.flatten())

    row_names = [f[0:2] for f in files]
    pd.DataFrame(np.array(letters)).to_csv("inputs.csv", header=None, index=None) 
##############################################

class Madaline():
    def __init__(self, conjunto_entradas, targets, taxa_aprendizagem, parada, ciclos_max, 
                 erro_tolerado, classes, **kwargs):
        
        self.numero_entradas = len(conjunto_entradas[0])
        self.conjunto_entradas = np.array(conjunto_entradas)
        self.targets = np.array(targets)
        self.classes = np.array(classes)
        self.taxa_aprendizagem = taxa_aprendizagem
        self.ciclos = 0
        self.ciclos_max = ciclos_max
        self.parada = parada
        self.erro_tolerado = erro_tolerado
        self.pesos = self.__inicializa_pesos()
        self.bias = np.array([random.uniform(-0.5,0.5) for saida in targets[0]])
        self.erro = 0
        
    def __inicializa_pesos(self):
        pesos = []
        for i in range(self.numero_entradas):
            pesos_entrada = []
            for j in range(len(self.targets[0])):
                pesos_entrada.append(random.uniform(-0.5,0.5))
            pesos.append(pesos_entrada)
        return np.array(pesos)
    
    def __f_ativacao(self, yin):
        out = np.copy(yin)
        out[out >= 0] = int(1)
        out[out < 0] = int(0)
        return out
    
    def treinamento(self):
        # x = [21,63], w = [63,7], b = [1,7], t = [21,7], y =x*w = [21,7]
        erros = []
        treinada = False
        self.ciclos = 1;
        while not treinada:
            self.erro = 0   
            yin = self.conjunto_entradas @ self.pesos + self.bias
            y = self.__f_ativacao(yin)
            self.erro = 0.5 * np.sum(np.power((self.targets - y), 2))
            # self.erro = self.erro_tolerado * np.sum(np.power((self.targets - y), 2))
            self.pesos = self.pesos + (self.taxa_aprendizagem * ((self.targets - y).T @ self.conjunto_entradas)).T
            self.bias = self.bias + (self.taxa_aprendizagem * np.sum((self.targets - y), axis=0))
            
            erros.append(self.erro)
            if (self.parada == 'ciclos' and self.ciclos >= self.ciclos_max) \
                or (self.parada == 'erro' and self.erro <= self.erro_tolerado):
                treinada = True
            else:
                self.ciclos += 1
        return list(range(1, self.ciclos+1)), erros

def processar():
    #display(os.getcwd())
    conjunto_entradas = pd.read_csv('inputs.csv', header=None).values
    targets_dataframe = pd.read_csv('targets.csv')
    targets = targets_dataframe.values
    classes = list(targets_dataframe.columns)
    
    madaline = Madaline(
        conjunto_entradas = conjunto_entradas, 
        targets = targets, 
        taxa_aprendizagem = taxa_aprendizagem.value,
        parada = parada.value.lower(),
        ciclos_max = ciclos.value,
        erro_tolerado = erro.value,
        classes = classes
    )
    
    lista_ciclos, lista_erros = madaline.treinamento()
    
    # salvando pesos e biases para ser possivel acessa-los na proxima celula
    pd.DataFrame(np.array(madaline.pesos)).to_csv("~pesos.csv", header=None, index=None) 
    pd.DataFrame(np.array(madaline.bias)).to_csv("~biases.csv", header=None, index=None) 
    
    ####### plota grafico #######
    plt.rcParams["figure.figsize"] = [10,6]
    plt.plot(lista_ciclos, lista_erros, 'r', label="Erro", marker='o', scalex=True)
    plt.style.use('ggplot')
    plt.title('Erro x Ciclo')
    plt.xlabel('Ciclo')
    plt.ylabel('Erro')
    for x,y in zip(lista_ciclos, lista_erros):
        plt.annotate(s='{:3.2f}'.format(y), xy=(x+0.1,y+0.1))
    plt.legend()
    plt.show()
    #############################
    


widgets.interact_manual.opts['manual_name'] = 'Treinar' # muda texto do botao
interact_manual(processar); # metodo a executar quando pressionar o botao

FloatSlider(value=0.5, continuous_update=False, description='Taxa de aprendizagem:', max=1.0, min=0.001, step=…

RadioButtons(description='Critério de parada:', options=('Erro', 'Ciclos'), style=DescriptionStyle(description…

IntSlider(value=10, continuous_update=False, description='Número de ciclos:', max=50, min=1, style=SliderStyle…

FloatSlider(value=0.1, continuous_update=False, description='Erro tolerado:', step=0.001, style=SliderStyle(de…

interactive(children=(Button(description='Treinar', style=ButtonStyle()), Output()), _dom_classes=('widget-int…

In [2]:
%%html
<style>
textarea
{
font-family: monospace;
}
</style>

In [3]:
letras_df = pd.read_csv('inputs.csv', header=None)
letras = letras_df.values

targets_dataframe = pd.read_csv('targets.csv')
targets = targets_dataframe.values

classes = np.array(targets_dataframe.columns)

letras_nome = ['A1', 'A2', 'A3',
             'B1', 'B2', 'B3',
             'C1', 'C2', 'C3',
             'D1', 'D2', 'D3',
             'E1', 'E2', 'E3',
             'J1', 'J2', 'J3',
             'K1', 'K2', 'K3']

def letras_to_char_array():
    letras_area = []
    for letra in letras:
        l = np.copy(letra)
        l = np.array(list(map(str, l)))
        l[l == '0'] = '.'
        l[l == '1'] = '#'
        letras_area.append(l.reshape(9,7))
    return letras_area

def char_array_to_str(letra):
    s = ''
    for linha in letra:
        for char in linha:
            s += char
        s += '\n'
    return s

def str_to_array(letra):
    ar = []
    for c in letra:
        if c == '.':
            ar.append(0)
        elif c == '#':
            ar.append(1)
    return np.array(ar)

letras_char = letras_to_char_array()

##################################
letra = widgets.Dropdown(
    options=letras_nome,
    description='Letra:',
    disabled=False,
)
display(letra)

l = Layout(flex='0 1 auto', height='150px', min_height='150px', width='300px')
area = widgets.Textarea(
    description='      ',
    value=char_array_to_str(letras_char[0]),
    layout = l,
    disabled=False
)
display(area)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        area.value = char_array_to_str(letras_char[letras_nome.index(change['new'])])

letra.observe(on_change)

###################################
def f_ativacao(yin):
    out = np.copy(yin)
    out[out >= 0] = 1
    out[out < 0] = 0
    return out

def operacao(letra, pesos, biases, classes):
    yin = letra @ pesos + biases
    y = f_ativacao(yin)
    return y, classes[y == 1][0] if len(classes[y == 1]) > 0 else None

def processar_teste():
    pesos = pd.read_csv('~pesos.csv', header=None).values
    biases = pd.read_csv('~biases.csv', header=None).values.flatten()

    y, classe = operacao(str_to_array(area.value), pesos, biases, classes)
    
    if classe is not None:
        saida = "<h3>Classe: <strong>{}</strong></h3>".format(classe)
    else:
        saida = "<h3>Erro! Não foi possível classificar!</h3>".format(classe)
    display(HTML(saida))
    
widgets.interact_manual.opts['manual_name'] = 'Testar' # muda texto do botao
interact_manual(processar_teste); # metodo a executar quando pressionar o botao

Dropdown(description='Letra:', options=('A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3', 'D1', 'D2', 'D3'…

Textarea(value='..##...\n...#...\n...#...\n..#.#..\n..#.#..\n.#####.\n.#...#.\n.#...#.\n###.###\n', descriptio…

interactive(children=(Button(description='Testar', style=ButtonStyle()), Output()), _dom_classes=('widget-inte…